## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1338]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from pprint import pprint

In [1339]:
import platform
from matplotlib import rc

# 운영 체제에 따라 폰트 설정
if platform.system() == 'Windows':  # Windows 환경
    rc('font', family='Malgun Gothic')  # 맑은 고딕

# 음수 표시 가능하도록 설정
plt.rcParams['axes.unicode_minus'] = False

반복적으로 쓰는 함수툴

In [1340]:
def plot_box(df, col_name):
    """
    주어진 데이터프레임과 열 이름에 대해 박스 플롯을 그리는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    """
    plt.figure(figsize=(7, 4))
    plt.boxplot(df[col_name], vert=False)
    plt.xlabel(col_name)
    plt.title(f'Box Plot of {col_name}')
    plt.show()

def value_counts_ratio(df, col_name, target_name='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열에 대해 각 값마다 타겟 변수의 비율과 갯수, 총 갯수를 출력하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    col_name (str): 열 이름
    target_name (str): 타겟 변수 이름
    """
    # 각 값마다 타겟 변수의 비율 계산
    value_counts = df.groupby(col_name)[target_name].value_counts(normalize=True).unstack().fillna(0)
    
    # 각 값마다 타겟 변수의 갯수 계산
    counts = df.groupby(col_name)[target_name].value_counts().unstack().fillna(0)
    
    # 각 값마다 총 갯수 계산
    total_counts = df[col_name].value_counts().rename('Total_Count')
    
    # 비율과 갯수를 합침
    result = value_counts.join(counts, lsuffix='_ratio', rsuffix='_count')
    
    # 총 갯수를 합침
    result = result.join(total_counts, on=col_name)
    
    # 출력 형식 조정
    result.index.name = 'variable'
    print(f"\n{col_name}별 {target_name} 비율 및 갯수\n")
    print(result.rename(columns=lambda x: x.split('_')[0]))

def summarize_group(df, group_by_columns):
    """
    주어진 데이터프레임을 특정 열들로 그룹화하여 각 그룹의 '임신_성공_여부' 비율과 갯수를 요약하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_columns (list): 그룹화할 열 이름들의 리스트
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_columns)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '임신 성공' 비율과 갯수 계산
        success_count = group['임신_성공_여부'].value_counts().get(1, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', "'임신_성공_여부' count", 'ratio', 'Total'])
    
    # 그룹화된 변수들의 이름을 제목행으로 출력
    print(f"Grouped by: {', '.join(group_by_columns)}")
    print()
    # 데이터프레임 출력
    print(results_df)

def plot_ratio(df, group_by_column, target_column='임신_성공_여부', success_value=1):
    """
    주어진 데이터프레임을 특정 열로 그룹화하여 각 그룹의 타겟 변수의 성공 비율을 시각화하는 함수.

    Parameters:
    df (pd.DataFrame): 데이터프레임
    group_by_column (str): 그룹화할 열 이름
    target_column (str): 타겟 변수 이름
    success_value (int): 타겟 변수의 성공을 나타내는 값
    """
    # 데이터프레임을 그룹화
    grouped_df = df.groupby(group_by_column)
    
    # 결과를 저장할 리스트 초기화
    results = []
    
    # 그룹화된 데이터프레임의 내용을 확인하는 코드
    for name, group in grouped_df:
        # 그룹의 갯수 계산
        group_count = group.shape[0]
        
        # '임신_성공_여부' 변수의 '성공' 비율과 갯수 계산
        success_count = group[target_column].value_counts().get(success_value, 0)
        success_ratio = success_count / group_count
        
        # 결과 리스트에 추가
        results.append([name, success_count, success_ratio, group_count])
    
    # 결과 리스트를 데이터프레임으로 변환
    results_df = pd.DataFrame(results, columns=['group', f"'{success_value}' count", 'ratio', 'Total'])
    
    # 그래프 크기 설정
    plt.figure(figsize=(10, 5))
    
    # 막대 그래프 생성
    ax = results_df.plot(kind='bar', x='group', y='ratio', legend=False)
    
    # 각 막대 위에 임신 성공 여부 갯수와 총 갯수 표시
    for i, (success_count, total) in enumerate(zip(results_df[f"'{success_value}' count"], results_df['Total'])):
        ax.text(i, results_df['ratio'][i], f'{success_count} ({total})', ha='center', va='bottom', fontsize=8)
    
    # 그래프 제목 및 축 레이블 설정
    ax.set_title(f'{success_value} Ratio by {group_by_column}')
    ax.set_xlabel(group_by_column)
    ax.set_ylabel(f'{success_value} Ratio')
   
    # 그래프 출력
    plt.show()

def plot_ratio_boxplot(data, column_name, target_column='임신_성공_여부'):
    """
    주어진 데이터프레임의 특정 열과 타겟 변수 간의 관계를 박스 플롯으로 시각화하는 함수.

    Parameters:
    data (pd.DataFrame): 데이터프레임
    column_name (str): 열 이름
    target_column (str): 타겟 변수 이름
    """

    # 그래프 그리기
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=column_name, y=target_column, data=data)

    # 그래프 제목 및 레이블 설정
    plt.title(f'{column_name} vs {target_column}')
    plt.xlabel(column_name)
    plt.ylabel(target_column)

    # 그래프 출력
    plt.show()

### Data Load

In [1341]:
# 데이터 로드
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# 변수명에서 공백을 밑줄로 대체
train.columns = train.columns.str.replace(' ', '_')
test.columns = test.columns.str.replace(' ', '_')

# 변경된 변수명 확인
print(train.columns)

Index(['ID', '시술_시기_코드', '시술_당시_나이', '임신_시도_또는_마지막_임신_경과_연수', '시술_유형',
       '특정_시술_유형', '배란_자극_여부', '배란_유도_유형', '단일_배아_이식_여부', '착상_전_유전_검사_사용_여부',
       '착상_전_유전_진단_사용_여부', '남성_주_불임_원인', '남성_부_불임_원인', '여성_주_불임_원인',
       '여성_부_불임_원인', '부부_주_불임_원인', '부부_부_불임_원인', '불명확_불임_원인', '불임_원인_-_난관_질환',
       '불임_원인_-_남성_요인', '불임_원인_-_배란_장애', '불임_원인_-_여성_요인', '불임_원인_-_자궁경부_문제',
       '불임_원인_-_자궁내막증', '불임_원인_-_정자_농도', '불임_원인_-_정자_면역학적_요인',
       '불임_원인_-_정자_운동성', '불임_원인_-_정자_형태', '배아_생성_주요_이유', '총_시술_횟수',
       '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수',
       'DI_임신_횟수', '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수', '총_생성_배아_수',
       '미세주입된_난자_수', '미세주입에서_생성된_배아_수', '이식된_배아_수', '미세주입_배아_이식_수', '저장된_배아_수',
       '미세주입_후_저장된_배아_수', '해동된_배아_수', '해동_난자_수', '수집된_신선_난자_수', '저장된_신선_난자_수',
       '혼합된_난자_수', '파트너_정자와_혼합된_난자_수', '기증자_정자와_혼합된_난자_수', '난자_출처', '정자_출처',
       '난자_기증자_나이', '정자_기증자_나이', '동결_배아_사용_여부', '신선_배아_사용_여부', '기증_배아_사용_여부',
       '대리모_여부', 'PGD_시술_여부', 'PGS_시술_여부

In [1342]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 69 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256351 non-null  object 
 1   시술_시기_코드               256351 non-null  object 
 2   시술_당시_나이               256351 non-null  object 
 3   임신_시도_또는_마지막_임신_경과_연수  9370 non-null    float64
 4   시술_유형                  256351 non-null  object 
 5   특정_시술_유형               256349 non-null  object 
 6   배란_자극_여부               256351 non-null  int64  
 7   배란_유도_유형               256351 non-null  object 
 8   단일_배아_이식_여부            250060 non-null  float64
 9   착상_전_유전_검사_사용_여부       2718 non-null    float64
 10  착상_전_유전_진단_사용_여부       250060 non-null  float64
 11  남성_주_불임_원인             256351 non-null  int64  
 12  남성_부_불임_원인             256351 non-null  int64  
 13  여성_주_불임_원인             256351 non-null  int64  
 14  여성_부_불임_원인             256351 non-nu

In [1343]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 68 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     90067 non-null  object 
 1   시술_시기_코드               90067 non-null  object 
 2   시술_당시_나이               90067 non-null  object 
 3   임신_시도_또는_마지막_임신_경과_연수  3297 non-null   float64
 4   시술_유형                  90067 non-null  object 
 5   특정_시술_유형               90067 non-null  object 
 6   배란_자극_여부               90067 non-null  int64  
 7   배란_유도_유형               90067 non-null  object 
 8   단일_배아_이식_여부            87891 non-null  float64
 9   착상_전_유전_검사_사용_여부       933 non-null    float64
 10  착상_전_유전_진단_사용_여부       87891 non-null  float64
 11  남성_주_불임_원인             90067 non-null  int64  
 12  남성_부_불임_원인             90067 non-null  int64  
 13  여성_주_불임_원인             90067 non-null  int64  
 14  여성_부_불임_원인             90067 non-null  int64  
 15  부부

In [1344]:
# '임신_성공_여부' 변수의 값 개수 계산
value_counts = train['임신_성공_여부'].value_counts()

# 각 값이 전체에서 차지하는 비율 계산
value_percentages = value_counts / len(train) * 100

# 결과 출력
print("임신_성공_여부 값 개수:")
print(value_counts)
print("\n임신_성공_여부 값 비율 (%):")
print(value_percentages)

임신_성공_여부 값 개수:
임신_성공_여부
0    190123
1     66228
Name: count, dtype: int64

임신_성공_여부 값 비율 (%):
임신_성공_여부
0    74.16511
1    25.83489
Name: count, dtype: float64


무조건 불임인 행 따로 구분해둠

In [1345]:
# # '시술_당시_나이' 변수의 값이 '알 수 없음'인 행 제거
# train_sub = train[train['시술_당시_나이'] == '알 수 없음']
# train = train[train['시술_당시_나이'] != '알 수 없음']

# # '남성_주_불임_원인'과 '남성_부_불임_원인'의 값이 동시에 1인 행 제거
# train_sub = pd.concat([train_sub, train[(train['남성_주_불임_원인'] == 1) & (train['남성_부_불임_원인'] == 1)]])
# train = train[~((train['남성_주_불임_원인'] == 1) & (train['남성_부_불임_원인'] == 1))]

# # '여성_주_불임_원인'과 '여성_부_불임_원인'의 값이 동시에 1인 행 제거
# train_sub = pd.concat([train_sub, train[(train['여성_주_불임_원인'] == 1) & (train['여성_부_불임_원인'] == 1)]])
# train = train[~((train['여성_주_불임_원인'] == 1) & (train['여성_부_불임_원인'] == 1))]

# # '부부_주_불임_원인'과 '부부_부_불임_원인'의 값이 동시에 1인 행 제거
# train_sub = pd.concat([train_sub, train[(train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1)]])
# train = train[~((train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1))]

In [1346]:
# 조건 정의
train_condition = (
    (train['시술_당시_나이'] == '알 수 없음') |
    ((train['남성_주_불임_원인'] == 1) & (train['남성_부_불임_원인'] == 1)) |
    ((train['여성_주_불임_원인'] == 1) & (train['여성_부_불임_원인'] == 1)) |
    ((train['부부_주_불임_원인'] == 1) & (train['부부_부_불임_원인'] == 1))
)

test_condition = (
    (test['시술_당시_나이'] == '알 수 없음') |
    ((test['남성_주_불임_원인'] == 1) & (test['남성_부_불임_원인'] == 1)) |
    ((test['여성_주_불임_원인'] == 1) & (test['여성_부_불임_원인'] == 1)) |
    ((test['부부_주_불임_원인'] == 1) & (test['부부_부_불임_원인'] == 1))
)

# ID 변수와 임신_성공_여부 변수를 제외한 나머지 열의 값을 -10으로 변경
train.loc[train_condition, train.columns.difference(['ID', '임신_성공_여부'])] = -10
test.loc[test_condition, test.columns.difference(['ID', '임신_성공_여부'])] = -10

## EDA

### 1. 시술_시기_코드

난임 시술을 받은 시기를 기준으로 코드 부여

In [1347]:
value_counts_ratio(train, '시술_시기_코드')


시술_시기_코드별 임신_성공_여부 비율 및 갯수

                 0         1      0     1  Total
variable                                        
-10       0.994169  0.005831    341     2    343
TRCMWS    0.742198  0.257802  28228  9805  38033
TRDQAZ    0.754095  0.245905  29329  9564  38893
TRJXFG    0.733306  0.266694  26377  9593  35970
TRVNRY    0.739976  0.260024  26742  9397  36139
TRXQMD    0.743715  0.256285  25885  8920  34805
TRYBLT    0.730480  0.269520  26775  9879  36654
TRZKPL    0.744664  0.255336  26446  9068  35514


In [1348]:
# '시술_시기_코드' 변수 제거
train = train.drop(columns=['시술_시기_코드'])
test = test.drop(columns=['시술_시기_코드'])

### 2. 시술 당시 나이

환자의 시술 당시 나이(연령대)

In [1349]:
value_counts_ratio(train, '시술_당시_나이')


시술_당시_나이별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
-10       0.994169  0.005831    341      2     343
만18-34세   0.677378  0.322622  69415  33061  102476
만35-37세   0.721585  0.278415  41691  16086   57777
만38-39세   0.782849  0.217151  30719   8521   39240
만40-42세   0.840599  0.159401  31393   5953   37346
만43-44세   0.882050  0.117950  10806   1445   12251
만45-50세   0.832321  0.167679   5758   1160    6918


In [1350]:
# '시술_당시_나이' 변수의 값을 중앙값으로 변환하는 함수 정의
def convert_age_category(age_category):
    if age_category == '만18-34세':
        return (18 + 34) / 2
    elif age_category == '만35-37세':
        return (35 + 37) / 2
    elif age_category == '만38-39세':
        return (38 + 39) / 2
    elif age_category == '만40-42세':
        return (40 + 42) / 2
    elif age_category == '만43-44세':
        return (43 + 44) / 2
    elif age_category == '만45-50세':
        return (45 + 50) / 2
    # elif age_category == '알 수 없음':
        # return -1
    else:
        return None

train['시술_당시_나이'] = train['시술_당시_나이'].apply(convert_age_category)
test['시술_당시_나이'] = test['시술_당시_나이'].apply(convert_age_category)

In [1351]:
train['시술_당시_나이'].value_counts()

시술_당시_나이
26.0    102476
36.0     57777
38.5     39240
41.0     37346
43.5     12251
47.5      6918
Name: count, dtype: int64

### 3. 임신 시도 또는 마지막 임신 경과 일수

환자가 처음 임신을 시도한 시점 또는 마지막 임신 이후 현재까지의 경과 연수 (년 단위)

In [1352]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1    0    1  Total
variable                                     
-10.0     0.994169  0.005831  341    2    343
 0.0      0.666667  0.333333    8    4     12
 1.0      0.666667  0.333333    4    2      6
 2.0      0.809524  0.190476   17    4     21
 3.0      0.752809  0.247191   67   22     89
 4.0      0.756579  0.243421  230   74    304
 5.0      0.753623  0.246377  520  170    690
 6.0      0.750253  0.249747  742  247    989
 7.0      0.763636  0.236364  924  286   1210
 8.0      0.784864  0.215136  923  253   1176
 9.0      0.796477  0.203523  814  208   1022
 10.0     0.778247  0.221753  737  210    947
 11.0     0.810271  0.189729  568  133    701
 12.0     0.780392  0.219608  398  112    510
 13.0     0.837838  0.162162  403   78    481
 14.0     0.776435  0.223565  257   74    331
 15.0     0.791339  0.208661  201   53    254
 16.0     0.846535  0.153465  171   31    202
 17.0     0.820144  0.179856  114   25

In [1353]:
print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

train['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)
test['임신_시도_또는_마지막_임신_경과_연수'].fillna(-1, inplace=True)

print("결측값 개수:", train['임신_시도_또는_마지막_임신_경과_연수'].isnull().sum())

결측값 개수: 246649
결측값 개수: 0


In [1354]:
value_counts_ratio(train, '임신_시도_또는_마지막_임신_경과_연수')


임신_시도_또는_마지막_임신_경과_연수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
-1.0      0.739727  0.260273  182453  64196  246649
 0.0      0.666667  0.333333       8      4      12
 1.0      0.666667  0.333333       4      2       6
 2.0      0.809524  0.190476      17      4      21
 3.0      0.752809  0.247191      67     22      89
 4.0      0.756579  0.243421     230     74     304
 5.0      0.753623  0.246377     520    170     690
 6.0      0.750253  0.249747     742    247     989
 7.0      0.763636  0.236364     924    286    1210
 8.0      0.784864  0.215136     923    253    1176
 9.0      0.796477  0.203523     814    208    1022
 10.0     0.778247  0.221753     737    210     947
 11.0     0.810271  0.189729     568    133     701
 12.0     0.780392  0.219608     398    112     510
 13.0     0.837838  0.162162     403     78     481
 14.0     0.776435  0.

### 4. 시술_유형

IVF 또는 DI 시술 여부

In [1355]:
value_counts_ratio(train, '시술_유형')


시술_유형별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
DI        0.871065  0.128935    5479    811    6290
IVF       0.738045  0.261955  184303  65415  249718


In [1356]:
test['시술_유형'].value_counts()

시술_유형
IVF    87769
DI      2176
-10      122
Name: count, dtype: int64

In [1357]:
# 시술_유형에 따라 파생변수 생성
train['시술_유형_IVF'] = np.where(train['시술_유형'] == 'IVF', 1, 0)
train['시술_유형_DI'] = np.where(train['시술_유형'] == 'DI', 1, 0)

test['시술_유형_IVF'] = np.where(test['시술_유형'] == 'IVF', 1, 0)
test['시술_유형_DI'] = np.where(test['시술_유형'] == 'DI', 1, 0)

In [1358]:
summarize_group(train, ['시술_유형_DI','시술_유형_IVF'])

Grouped by: 시술_유형_DI, 시술_유형_IVF

    group  '임신_성공_여부' count     ratio   Total
0  (0, 0)                 2  0.005831     343
1  (0, 1)             65415  0.261955  249718
2  (1, 0)               811  0.128935    6290


In [1359]:
train = train.drop(columns=['시술_유형'])
test = test.drop(columns=['시술_유형'])

### 5. 특정_시술_유형

IVF – 체외 수정 (시험관 아기 시술)  
ICSI – 세포질 내 정자 주입  
IUI – 인공 수정 (자궁 내 주입)  
ICI – 자궁경부 인공 수정  
GIFT – 생식세포 나팔관 이식  
FER – 냉동 배아 이식  
Generic DI – 기증 정자 주입  
IVI – 질 내 인공 수정   
BLASTOCYST – 배반포 단계에 이식  
AH – 배아를 hatching 시켜주는 과정  
Unknown – 시술 유형 정보가 제공되지 않았거나 알 수 없음  

(*) 특수 기호의 의미  
"/" – 복합적인 시술 방법을 의미 (예: ICSI / BLASTOCYST)  
":" – 동일 시술 내 세부 조합을 의미 (예: ICSI:IVF)  

In [1360]:
value_counts_ratio(train, '특정_시술_유형')


특정_시술_유형별 임신_성공_여부 비율 및 갯수

                                            0         1        0        1  \
variable                                                                    
-10                                  0.994169  0.005831    341.0      2.0   
FER                                  0.666667  0.333333      2.0      1.0   
GIFT                                 1.000000  0.000000      2.0      0.0   
Generic DI                           0.830986  0.169014     59.0     12.0   
ICI                                  0.864583  0.135417     83.0     13.0   
ICSI                                 0.727137  0.272863  88966.0  33385.0   
ICSI / AH                            0.782835  0.217165    602.0    167.0   
ICSI / AH:Unknown                    0.500000  0.500000      1.0      1.0   
ICSI / BLASTOCYST                    0.643257  0.356743   1035.0    574.0   
ICSI / BLASTOCYST :ICSI              0.000000  1.000000      0.0      1.0   
ICSI / BLASTOCYST :IVF / BLASTOCYST  0.750000  

In [1361]:
test['특정_시술_유형'].value_counts()

특정_시술_유형
ICSI                                  43228
IVF                                   32243
Unknown                                9213
IUI                                    2112
ICSI:ICSI                               796
ICSI / BLASTOCYST                       584
IVF:IVF                                 405
IVF / BLASTOCYST                        388
ICSI:IVF                                298
ICSI / AH                               264
IVF:ICSI                                133
-10                                     122
IVF / AH                                105
ICSI:Unknown                             68
IVF:Unknown                              41
ICI                                      28
Generic DI                               26
IVI                                       8
ICSI / BLASTOCYST:IVF / BLASTOCYST        2
IUI:ICI                                   2
IVF:Unknown:Unknown:Unknown               1
Name: count, dtype: int64

In [1362]:
# '특정_시술_유형' 변수의 값이 'FER' 또는 'GIFT'인 행 제거
train = train.loc[~train['특정_시술_유형'].isin(['FER', 'GIFT'])]
# test = test.loc[~test['특정_시술_유형'].isin(['FER', 'GIFT'])]

In [1363]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 시술 유형 수정 함수 정의
def clean_procedure(procedure):
    if isinstance(procedure, str):
        # 공백 제거
        procedure = procedure.replace(' ', '')
        # 시술 뒤에 Unknown이 붙어있으면 제거
        if ':Unknown' in procedure:
            procedure = procedure.replace(':Unknown', '')
    return procedure

# 시술 유형 수정
train['특정_시술_유형'] = train['특정_시술_유형'].apply(clean_procedure)
test['특정_시술_유형'] = test['특정_시술_유형'].apply(clean_procedure)

# 시술 유형 분리 및 원핫 인코딩 함수 정의
def process_procedures(df):
    # 시술 유형 분리
    df_split = df['특정_시술_유형'].apply(lambda x: x.split(':') if isinstance(x, str) else [])
    
    # '/'로 구분된 시술 유형을 개별 항목으로 분리
    df_split = df_split.apply(lambda x: [item for sublist in [i.split('/') for i in x] for item in sublist])
    
    # MultiLabelBinarizer를 사용하여 원핫 인코딩
    mlb = MultiLabelBinarizer()
    reasons_encoded = mlb.fit_transform(df_split)
    reasons_df = pd.DataFrame(reasons_encoded, columns=[f'특정_시술_유형_{cls}' for cls in mlb.classes_])
    
    # '특정_시술_유형_Unknown' 변수 제거
    if '특정_시술_유형_Unknown' in reasons_df.columns:
        reasons_df = reasons_df.drop(columns=['특정_시술_유형_Unknown'])
    
    # 원래 데이터프레임과 병합
    df_encoded = df.drop(columns=['특정_시술_유형']).reset_index(drop=True)
    df_encoded = pd.concat([df_encoded, reasons_df], axis=1)
    
    return df_encoded

train = process_procedures(train)
test = process_procedures(test)

In [1364]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256346 entries, 0 to 256345
Data columns (total 76 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256346 non-null  object 
 1   시술_당시_나이               256003 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256346 non-null  float64
 3   배란_자극_여부               256346 non-null  int64  
 4   배란_유도_유형               256346 non-null  object 
 5   단일_배아_이식_여부            250057 non-null  float64
 6   착상_전_유전_검사_사용_여부       3061 non-null    float64
 7   착상_전_유전_진단_사용_여부       250057 non-null  float64
 8   남성_주_불임_원인             256346 non-null  int64  
 9   남성_부_불임_원인             256346 non-null  int64  
 10  여성_주_불임_원인             256346 non-null  int64  
 11  여성_부_불임_원인             256346 non-null  int64  
 12  부부_주_불임_원인             256346 non-null  int64  
 13  부부_부_불임_원인             256346 non-null  int64  
 14  불명확_불임_원인              256346 non-nu

### 6. 배란_자극_여부

배란 자극 치료가 사용되었는지 여부

In [1365]:
value_counts_ratio(train, '배란_자극_여부')


배란_자극_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.768344  0.231656   45038  13579   58617
 1        0.733284  0.266716  144740  52646  197386


### 7. 배란_유도_유형

사용된 배란 유도 방법 유형

In [1366]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                     0         1         0        1   Total
variable                                                   
-10           0.994169  0.005831     341.0      2.0     343
기록되지 않은 시행    0.731133  0.268867  141913.0  52187.0  194100
생식선 자극 호르몬    0.000000  1.000000       0.0      1.0       1
세트로타이드 (억제제)  1.000000  0.000000       1.0      0.0       1
알 수 없음        0.773235  0.226765   47864.0  14037.0   61901


In [1367]:
test['배란_유도_유형'].value_counts()

배란_유도_유형
기록되지 않은 시행    68492
알 수 없음        21453
-10             122
Name: count, dtype: int64

In [1368]:
# 해당하는 행이 적은 '생식선 자극 호르몬', '세트로타이드 (억제제)' 제거
train = train.loc[~train['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]
test = test.loc[~test['배란_유도_유형'].isin(['생식선 자극 호르몬', '세트로타이드 (억제제)'])]

In [1369]:
value_counts_ratio(train, '배란_유도_유형')


배란_유도_유형별 임신_성공_여부 비율 및 갯수

                   0         1       0      1   Total
variable                                             
-10         0.994169  0.005831     341      2     343
기록되지 않은 시행  0.731133  0.268867  141913  52187  194100
알 수 없음      0.773235  0.226765   47864  14037   61901


In [1370]:
# '배란_유도_유형' 변수 제거
train = train.drop(columns=['배란_유도_유형'])
test = test.drop(columns=['배란_유도_유형'])

### 8. 단일_배아_이식_여부

선택적으로 단일 배아를 이식했는지 여부

In [1371]:
value_counts_ratio(train, '단일_배아_이식_여부')


단일_배아_이식_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.770172  0.229828  147357  43973  191330
 1.0      0.632764  0.367236   36942  21440   58382


In [1372]:
# 결측값 개수 확인
print("결측값 개수:", train['단일_배아_이식_여부'].isnull().sum())

# 결측값을 -1 으로 채우기
train['단일_배아_이식_여부'].fillna(-1, inplace=True)
test['단일_배아_이식_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['단일_배아_이식_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 9. 착상_전_유전_검사_사용_여부

PGS(착상 전 유전 검사)가 시행되었는지 여부

In [1373]:
value_counts_ratio(train, '착상_전_유전_검사_사용_여부')


착상_전_유전_검사_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
-10.0     0.994169  0.005831   341    2    343
 1.0      0.851729  0.148271  2315  403   2718


In [1374]:
# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)
test['착상_전_유전_검사_사용_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_검사_사용_여부'].isnull().sum())

결측값 개수: 253283
결측값 개수: 0


In [1375]:
value_counts_ratio(train, '착상_전_유전_검사_사용_여부')


착상_전_유전_검사_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.740129  0.259871  187462  65821  253283
 1.0      0.851729  0.148271    2315    403    2718


### 10. 착상_전_유전_진단_사용_여부

PGD(착상 전 유전 진단)가 시행되었는지 여부

In [1376]:
value_counts_ratio(train, '착상_전_유전_진단_사용_여부')


착상_전_유전_진단_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.737141  0.262859  181717  64799  246516
 1.0      0.807885  0.192115    2582    614    3196


In [1377]:
# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_진단_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['착상_전_유전_진단_사용_여부'].fillna(-1, inplace=True)
test['착상_전_유전_진단_사용_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['착상_전_유전_진단_사용_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


In [1378]:
value_counts_ratio(train, '착상_전_유전_진단_사용_여부')


착상_전_유전_진단_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
-1.0      0.871045  0.128955    5478    811    6289
 0.0      0.737141  0.262859  181717  64799  246516
 1.0      0.807885  0.192115    2582    614    3196


In [1379]:
summarize_group(train, ['착상_전_유전_진단_사용_여부', '착상_전_유전_검사_사용_여부'])

Grouped by: 착상_전_유전_진단_사용_여부, 착상_전_유전_검사_사용_여부

            group  '임신_성공_여부' count     ratio   Total
0  (-10.0, -10.0)                 2  0.005831     343
1     (-1.0, 0.0)               811  0.128955    6289
2      (0.0, 0.0)             64398  0.264117  243824
3      (0.0, 1.0)               401  0.148960    2692
4      (1.0, 0.0)               612  0.193060    3170
5      (1.0, 1.0)                 2  0.076923      26


### 11. 남성_주_불임_원인

남성의 주요 불임 원인 여부

In [1380]:
value_counts_ratio(train, '남성_주_불임_원인')


남성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.740177  0.259823  184082  64618  248700
 1        0.780030  0.219970    5695   1606    7301


### 12. 남성_부_불임_원인

남성의 부차적 불임 원인 여부

In [1381]:
value_counts_ratio(train, '남성_부_불임_원인')


남성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.740504  0.259496  187086  65561  252647
 1        0.802326  0.197674    2691    663    3354


In [1382]:
summarize_group(train, ['남성_주_불임_원인', '남성_부_불임_원인'])

Grouped by: 남성_주_불임_원인, 남성_부_불임_원인

        group  '임신_성공_여부' count     ratio   Total
0  (-10, -10)                 2  0.005831     343
1      (0, 0)             63955  0.260673  245346
2      (0, 1)               663  0.197674    3354
3      (1, 0)              1606  0.219970    7301


### 13. 여성_주_불임_원인

여성의 주요 불임 원인 여부

In [1383]:
value_counts_ratio(train, '여성_주_불임_원인')


여성_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.740057  0.259943  183634  64501  248135
 1        0.780956  0.219044    6143   1723    7866


### 14. 여성_부_불임_원인

여성의 부차적 불임 원인 여부

In [1384]:
value_counts_ratio(train, '여성_부_불임_원인')


여성_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.740526  0.259474  187225  65602  252827
 1        0.804033  0.195967    2552    622    3174


In [1385]:
summarize_group(train, ['여성_주_불임_원인', '여성_부_불임_원인'])

Grouped by: 여성_주_불임_원인, 여성_부_불임_원인

        group  '임신_성공_여부' count     ratio   Total
0  (-10, -10)                 2  0.005831     343
1      (0, 0)             63879  0.260772  244961
2      (0, 1)               622  0.195967    3174
3      (1, 0)              1723  0.219044    7866


### 15. 부부_주_불임_원인

부부의 주요 불임 원인 여부

In [1386]:
value_counts_ratio(train, '부부_주_불임_원인')


부부_주_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.740001  0.259999  183179  64360  247539
 1        0.779721  0.220279    6598   1864    8462


### 16. 부부_부_불임_원인

부부의 부차적 불임 원인 여부

In [1387]:
value_counts_ratio(train, '부부_부_불임_원인')


부부_부_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.740684  0.259316  187956  65804  253760
 1        0.812584  0.187416    1821    420    2241


In [1388]:
summarize_group(train, ['부부_주_불임_원인', '부부_부_불임_원인'])

Grouped by: 부부_주_불임_원인, 부부_부_불임_원인

        group  '임신_성공_여부' count     ratio   Total
0  (-10, -10)                 2  0.005831     343
1      (0, 0)             63940  0.260663  245298
2      (0, 1)               420  0.187416    2241
3      (1, 0)              1864  0.220279    8462


### 17. 불명확_불임_원인

불명의 원인으로 인한 불임 여부

In [1389]:
value_counts_ratio(train, '불명확_불임_원인')


불명확_불임_원인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741578  0.258422  142182  49547  191729
 1        0.740525  0.259475   47595  16677   64272


### 18. 불임_원인_-_난관_질환

난관 이상으로 인한 불임 여부

In [1390]:
value_counts_ratio(train, '불임_원인_-_난관_질환')


불임_원인_-_난관_질환별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741165  0.258835  163389  57060  220449
 1        0.742237  0.257763   26388   9164   35552


### 19. 불임_원인_-_남성_요인

남성 요인으로 인한 불임 여부

In [1391]:
value_counts_ratio(train, '불임_원인_-_남성_요인')


불임_원인_-_남성_요인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.753923  0.246077  121040  39507  160547
 1        0.720106  0.279894   68737  26717   95454


### 20. 불임_원인_-_배란_장애

배란 장애로 인한 불임 여부

In [1392]:
value_counts_ratio(train, '불임_원인_-_배란_장애')


불임_원인_-_배란_장애별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.745511  0.254489  165932  56643  222575
 1        0.713367  0.286633   23845   9581   33426


### 21. 불임_원인_-_여성_요인

여성 요인으로 인한 불임 여부

In [1393]:
value_counts_ratio(train, '불임_원인_-_여성_요인')


불임_원인_-_여성_요인별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741314  0.258686  189777  66224  256001


In [1394]:
test['불임_원인_-_여성_요인'].value_counts()

불임_원인_-_여성_요인
 0     89945
-10      122
Name: count, dtype: int64

In [1395]:
train['불임_원인_-_여성_요인'].value_counts()

불임_원인_-_여성_요인
 0     256001
-10       343
Name: count, dtype: int64

In [1396]:
train['불임_원인_-_여성_요인'].isnull().sum()

0

In [1397]:
# '불임_원인_-_여성_요인' 변수 제거
train = train.drop(columns=['불임_원인_-_여성_요인'])
test = test.drop(columns=['불임_원인_-_여성_요인'])

### 22. 불임_원인_-_자궁경부_문제

자궁경부 이상으로 인한 불임 여부

In [1398]:
value_counts_ratio(train, '불임_원인_-_자궁경부_문제')


불임_원인_-_자궁경부_문제별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
 0        0.741303  0.258697  189767.0  66224.0  255991
 1        1.000000  0.000000      10.0      0.0      10


### 23. 불임_원인_-_자궁내막증

자궁내막증으로 인한 불임 여부

In [1399]:
value_counts_ratio(train, '불임_원인_-_자궁내막증')


불임_원인_-_자궁내막증별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741017  0.258983  176139  61560  237699
 1        0.745164  0.254836   13638   4664   18302


### 24. 불임_원인_-_정자_농도

파트너의 정자 농도 문제 여부

In [1400]:
value_counts_ratio(train, '불임_원인_-_정자_농도')


불임_원인_-_정자_농도별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741304  0.258696  189570  66155  255725
 1        0.750000  0.250000     207     69     276


### 25. 불임_원인_-_정자_면역학적_요인

파트너의 정자 면역 관련 문제 여부

In [1401]:
value_counts_ratio(train, '불임_원인_-_정자_면역학적_요인')


불임_원인_-_정자_면역학적_요인별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
 0        0.741313  0.258688  189776.0  66224.0  256000
 1        1.000000  0.000000       1.0      0.0       1


In [1402]:
test['불임_원인_-_정자_면역학적_요인'].value_counts()

불임_원인_-_정자_면역학적_요인
 0     89944
-10      122
 1         1
Name: count, dtype: int64

In [1403]:
train = train.drop(columns=['불임_원인_-_정자_면역학적_요인'])
test = test.drop(columns=['불임_원인_-_정자_면역학적_요인'])

### 26. 불임_원인_-_정자_운동성

파트너의 정자 운동성 문제 여부

In [1404]:
value_counts_ratio(train, '불임_원인_-_정자_운동성')


불임_원인_-_정자_운동성별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741282  0.258718  189697  66207  255904
 1        0.824742  0.175258      80     17      97


### 27. 불임_원인_-_정자_형태

파트너의 정자 형태 문제 여부

In [1405]:
value_counts_ratio(train, '불임_원인_-_정자_형태')


불임_원인_-_정자_형태별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
 0        0.741294  0.258706  189666  66192  255858
 1        0.776224  0.223776     111     32     143


In [1406]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256344 entries, 0 to 256345
Data columns (total 73 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256001 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-null  i

In [1407]:
summarize_group(train, [
    '불임_원인_-_난관_질환',
    # '불임_원인_-_남성_요인',
    '불임_원인_-_배란_장애',
    '불임_원인_-_자궁경부_문제',
    '불임_원인_-_자궁내막증',
    # '불임_원인_-_정자_농도',
    # '불임_원인_-_정자_운동성',
    # '불임_원인_-_정자_형태'
])

Grouped by: 불임_원인_-_난관_질환, 불임_원인_-_배란_장애, 불임_원인_-_자궁경부_문제, 불임_원인_-_자궁내막증

                   group  '임신_성공_여부' count     ratio   Total
0   (-10, -10, -10, -10)                 2  0.005831     343
1           (0, 0, 0, 0)             44992  0.254396  176858
2           (0, 0, 0, 1)              3395  0.254574   13336
3           (0, 0, 1, 0)                 0  0.000000       7
4           (0, 1, 0, 0)              8281  0.286758   28878
5           (0, 1, 0, 1)               392  0.286340    1369
6           (0, 1, 1, 0)                 0  0.000000       1
7           (1, 0, 0, 0)              7455  0.256379   29078
8           (1, 0, 0, 1)               801  0.243169    3294
9           (1, 0, 1, 1)                 0  0.000000       2
10          (1, 1, 0, 0)               832  0.289190    2877
11          (1, 1, 0, 1)                76  0.252492     301


### 28. 배아_생성_주요_이유

배아 생성의 주된 목적(시술, 보관 등)

In [1408]:
value_counts_ratio(train, '배아_생성_주요_이유')


배아_생성_주요_이유별 임신_성공_여부 비율 및 갯수

                            0         1         0        1   Total
variable                                                          
-10                  0.994169  0.005831     341.0      2.0     343
기증용                  1.000000  0.000000     794.0      0.0     794
기증용, 난자 저장용          1.000000  0.000000      42.0      0.0      42
기증용, 배아 저장용          1.000000  0.000000     112.0      0.0     112
기증용, 배아 저장용, 현재 시술용  0.650000  0.350000      13.0      7.0      20
기증용, 현재 시술용          0.620243  0.379757    2347.0   1437.0    3784
난자 저장용               1.000000  0.000000    1959.0      0.0    1959
난자 저장용, 배아 저장용       1.000000  0.000000       6.0      0.0       6
난자 저장용, 배아 저장용, 연구용  1.000000  0.000000       1.0      0.0       1
난자 저장용, 현재 시술용       0.800000  0.200000       4.0      1.0       5
배아 저장용               0.999130  0.000870    9183.0      8.0    9191
배아 저장용, 현재 시술용       0.734940  0.265060      61.0     22.0      83
연구용, 현재 시술용          1.000000 

In [1409]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# 배아_생성_주요_이유 분리 함수 정의
def split_reasons(reason):
    if isinstance(reason, str):
        return reason.split(', ')
    return []

# MultiLabelBinarizer를 사용하여 원핫 인코딩
def process_reasons(df):
    mlb = MultiLabelBinarizer()
    reasons = df['배아_생성_주요_이유'].apply(split_reasons)
    reasons_encoded = mlb.fit_transform(reasons)
    reasons_df = pd.DataFrame(reasons_encoded, columns=[f'배아_생성_주요_이유_{cls}' for cls in mlb.classes_])
    df = df.drop(columns=['배아_생성_주요_이유']).reset_index(drop=True)
    df_encoded = pd.concat([df, reasons_df], axis=1)
    
    # '배아_생성_주요_이유_연구용' 변수 제거
    if '배아_생성_주요_이유_연구용' in df_encoded.columns:
        df_encoded = df_encoded.drop(columns=['배아_생성_주요_이유_연구용'])
    
    return df_encoded

train = process_reasons(train)
test = process_reasons(test)

In [1410]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 76 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256001 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

### 29. 총_시술_횟수

IVF 및 DI 포함한 전체 시술 횟수

In [1411]:
value_counts_ratio(train, '총_시술_횟수')


총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
-10       0.994169  0.005831    341      2    343
0회        0.708703  0.291297  69095  28400  97495
1회        0.749824  0.250176  42563  14201  56764
2회        0.755444  0.244556  29664   9603  39267
3회        0.763212  0.236788  18688   5798  24486
4회        0.771746  0.228254  11658   3448  15106
5회        0.783709  0.216291   7120   1965   9085
6회 이상     0.796420  0.203580  10989   2809  13798


### 30. 클리닉_내_총_시술_횟수

특정 클리닉에서 받은 IVF 및 DI 시술 횟수

In [1412]:
value_counts_ratio(train, '클리닉_내_총_시술_횟수')


클리닉_내_총_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
-10       0.994169  0.005831    341      2     343
0회        0.716840  0.283160  87147  34424  121571
1회        0.754410  0.245590  45036  14661   59697
2회        0.761612  0.238388  26268   8222   34490
3회        0.766983  0.233017  14045   4267   18312
4회        0.777422  0.222578   7761   2222    9983
5회        0.791550  0.208450   4253   1120    5373
6회 이상     0.801065  0.198935   5267   1308    6575


### 31. IVF_시술_횟수

이전에 수행된 IVF 시술 횟수

In [1413]:
value_counts_ratio(train, 'IVF_시술_횟수')


IVF_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
-10       0.994169  0.005831    341      2     343
0회        0.713455  0.286545  74078  29752  103830
1회        0.749211  0.250789  43667  14617   58284
2회        0.754149  0.245851  29491   9614   39105
3회        0.764020  0.235980  17752   5483   23235
4회        0.772037  0.227963  10519   3106   13625
5회        0.786660  0.213340   6121   1660    7781
6회 이상     0.803570  0.196430   8149   1992   10141


### 32. DI_시술_횟수

이전에 수행된 DI 시술 횟수

In [1414]:
value_counts_ratio(train, 'DI_시술_횟수')


DI_시술_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
0회        0.738989  0.261011  178921  63195  242116
1회        0.807877  0.192123    2359    561    2920
2회        0.782768  0.217232    2353    653    3006
3회        0.757702  0.242298    2361    755    3116
4회        0.779386  0.220614    1346    381    1727
5회        0.811667  0.188333     974    226    1200
6회 이상     0.763570  0.236430    1463    453    1916


### 33. 총_임신_횟수

IVF 및 DI 포함한 총 임신 횟수

In [1415]:
value_counts_ratio(train, '총_임신_횟수')


총_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
0회        0.742133  0.257867  152201  52885  205086
1회        0.737068  0.262932   32302  11523   43825
2회        0.742029  0.257971    4631   1610    6241
3회        0.754692  0.245308     563    183     746
4회        0.782609  0.217391      72     20      92
5회        0.750000  0.250000       6      2       8
6회 이상     0.666667  0.333333       2      1       3


### 34. IVF_임신_횟수

IVF 시술로 인한 임신 횟수

In [1416]:
value_counts_ratio(train, 'IVF_임신_횟수')


IVF_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
0회        0.742742  0.257258  154454.0  53497.0  207951
1회        0.733566  0.266434   30454.0  11061.0   41515
2회        0.743949  0.256051    4303.0   1481.0    5784
3회        0.752640  0.247360     499.0    164.0     663
4회        0.762500  0.237500      61.0     19.0      80
5회        0.857143  0.142857       6.0      1.0       7
6회 이상     0.000000  1.000000       0.0      1.0       1


### 35. DI_임신_횟수

DI 시술로 인한 임신 횟수

In [1417]:
value_counts_ratio(train, 'DI_임신_횟수')


DI_임신_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
0회        0.740810  0.259190  187390.0  65563.0  252953
1회        0.790857  0.209143    2076.0    549.0    2625
2회        0.724796  0.275204     266.0    101.0     367
3회        0.791667  0.208333      38.0     10.0      48
4회        1.000000  0.000000       5.0      0.0       5
5회        0.666667  0.333333       2.0      1.0       3


### 36. 총_출산_횟수

IVF 및 DI 시술로 출산한 횟수

In [1418]:
value_counts_ratio(train, '총_출산_횟수')


총_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
0회        0.742763  0.257237  162082.0  56133.0  218215
1회        0.732707  0.267293   25910.0   9452.0   35362
2회        0.736936  0.263064    1650.0    589.0    2239
3회        0.721893  0.278107     122.0     47.0     169
4회        0.769231  0.230769      10.0      3.0      13
5회        1.000000  0.000000       1.0      0.0       1
6회 이상     1.000000  0.000000       2.0      0.0       2


### 37. IVF_출산_횟수

IVF 시술로 출산한 횟수

In [1419]:
value_counts_ratio(train, 'IVF_출산_횟수')


IVF_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
0회        0.743072  0.256928  163840.0  56650.0  220490
1회        0.730062  0.269938   24341.0   9000.0   33341
2회        0.737494  0.262506    1489.0    530.0    2019
3회        0.700730  0.299270      96.0     41.0     137
4회        0.769231  0.230769      10.0      3.0      13
5회        1.000000  0.000000       1.0      0.0       1


### 38. DI_출산_횟수

DI 시술로 출산한 횟수

In [1420]:
value_counts_ratio(train, 'DI_출산_횟수')


DI_출산_횟수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10       0.994169  0.005831     341.0      2.0     343
0회        0.741043  0.258957  187973.0  65687.0  253660
1회        0.770814  0.229186    1648.0    490.0    2138
2회        0.763736  0.236264     139.0     43.0     182
3회        0.789474  0.210526      15.0      4.0      19
5회        1.000000  0.000000       2.0      0.0       2


In [1421]:
# 숫자 매핑
mapping = {
    '0회': 0, '1회': 1, '2회': 2, '3회': 3, 
    '4회': 4, '5회': 5, '6회 이상': 6
}

# 변환할 열 목록
columns_to_map = [
    '총_시술_횟수', '클리닉_내_총_시술_횟수', 'IVF_시술_횟수', 
    'DI_시술_횟수', '총_임신_횟수', 'IVF_임신_횟수', 'DI_임신_횟수', 
    '총_출산_횟수', 'IVF_출산_횟수', 'DI_출산_횟수'
]

# 숫자로 변환
for col in columns_to_map:
    train[col] = train[col].map(mapping)
    test[col] = test[col].map(mapping)

In [1422]:
# 비율 계산 (train 데이터프레임)
train['IVF_임신_시술_비율'] = np.where(train['IVF_시술_횟수'] > 0, train['IVF_임신_횟수'] / train['IVF_시술_횟수'], -1)
train['DI_임신_시술_비율'] = np.where(train['DI_시술_횟수'] > 0, train['DI_임신_횟수'] / train['DI_시술_횟수'], -1)
train['임신_성공률'] = np.where(train['총_시술_횟수'] > 0, train['총_임신_횟수'] / train['총_시술_횟수'], -1)

# 비율 계산 (test 데이터프레임)
test['IVF_임신_시술_비율'] = np.where(test['IVF_시술_횟수'] > 0, test['IVF_임신_횟수'] / test['IVF_시술_횟수'], -1)
test['DI_임신_시술_비율'] = np.where(test['DI_시술_횟수'] > 0, test['DI_임신_횟수'] / test['DI_시술_횟수'], -1)
test['임신_성공률'] = np.where(test['총_시술_횟수'] > 0, test['총_임신_횟수'] / test['총_시술_횟수'], -1)

### 39. 총_생성_배아_수

해당 시술에서 생성된 배아의 총 개수

In [1423]:
value_counts_ratio(train, '총_생성_배아_수')


총_생성_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
-10.0     0.994169  0.005831    341.0     2.0    343
 0.0      0.827181  0.172819  43872.0  9166.0  53038
 1.0      0.905106  0.094894  13773.0  1444.0  15217
 2.0      0.819878  0.180122  15731.0  3456.0  19187
 3.0      0.758104  0.241896  15880.0  5067.0  20947
 4.0      0.722649  0.277351  15615.0  5993.0  21608
 5.0      0.694502  0.305498  14172.0  6234.0  20406
 6.0      0.676978  0.323022  12606.0  6015.0  18621
 7.0      0.661591  0.338409  10602.0  5423.0  16025
 8.0      0.649974  0.350026   8852.0  4767.0  13619
 9.0      0.645579  0.354421   7184.0  3944.0  11128
 10.0     0.639523  0.360477   5796.0  3267.0   9063
 11.0     0.633145  0.366855   4489.0  2601.0   7090
 12.0     0.647552  0.352448   3704.0  2016.0   5720
 13.0     0.633796  0.366204   2738.0  1582.0   4320
 14.0     0.644048  0.355952   2164.0  1196.0   3360
 15.0     0.6422

In [1424]:
# 결측값 개수 확인
print("결측값 개수:", train['총_생성_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['총_생성_배아_수'].fillna(0, inplace=True)
test['총_생성_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['총_생성_배아_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 40. 미세주입된_난자_수

정자 미세주입(ICSI)으로 처리된 난자의 수

In [1425]:
value_counts_ratio(train, '미세주입된_난자_수')


미세주입된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
-10.0     0.994169  0.005831    341.0      2.0     343
 0.0      0.748636  0.251364  95332.0  32009.0  127341
 1.0      0.942643  0.057357   4273.0    260.0    4533
 2.0      0.888765  0.111235   5593.0    700.0    6293
 3.0      0.833397  0.166603   6593.0   1318.0    7911
 4.0      0.789731  0.210269   7598.0   2023.0    9621
 5.0      0.744609  0.255391   7907.0   2712.0   10619
 6.0      0.723316  0.276684   8010.0   3064.0   11074
 7.0      0.704746  0.295254   7335.0   3073.0   10408
 8.0      0.693648  0.306352   6727.0   2971.0    9698
 9.0      0.681697  0.318303   5866.0   2739.0    8605
 10.0     0.665462  0.334538   5156.0   2592.0    7748
 11.0     0.650268  0.349732   4243.0   2282.0    6525
 12.0     0.660559  0.339441   3713.0   1908.0    5621
 13.0     0.642012  0.357988   2986.0   1665.0    4651
 14.0     0.661374  0.338626   254

In [1426]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입된_난자_수'].fillna(0, inplace=True)
test['미세주입된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입된_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 41. 미세주입에서_생성된_배아_수

ICSI를 통해 생성된 배아의 수

In [1427]:
value_counts_ratio(train, '미세주입에서_생성된_배아_수')


미세주입에서_생성된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1   Total
variable                                              
-10.0     0.994169  0.005831    341.0      2.0     343
 0.0      0.756314  0.243686  99413.0  32031.0  131444
 1.0      0.908495  0.091505   9005.0    907.0    9912
 2.0      0.817894  0.182106  10330.0   2300.0   12630
 3.0      0.758276  0.241724  10468.0   3337.0   13805
 4.0      0.719689  0.280311   9990.0   3891.0   13881
 5.0      0.689522  0.310478   8910.0   4012.0   12922
 6.0      0.671845  0.328155   7784.0   3802.0   11586
 7.0      0.659308  0.340692   6272.0   3241.0    9513
 8.0      0.651130  0.348870   5071.0   2717.0    7788
 9.0      0.641491  0.358509   3958.0   2212.0    6170
 10.0     0.635817  0.364183   3064.0   1755.0    4819
 11.0     0.624231  0.375769   2334.0   1405.0    3739
 12.0     0.662865  0.337135   1976.0   1005.0    2981
 13.0     0.642027  0.357973   1381.0    770.0    2151
 14.0     0.646536  0.353464 

In [1428]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입에서_생성된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입에서_생성된_배아_수'].fillna(0, inplace=True)
test['미세주입에서_생성된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입에서_생성된_배아_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 42. 이식된_배아_수

이식된 배아의 총 수

In [1429]:
value_counts_ratio(train, '이식된_배아_수')


이식된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1      0      1   Total
variable                                          
-10.0     0.994169  0.005831    341      2     343
 0.0      0.999172  0.000828  36182     30   36212
 1.0      0.686446  0.313554  64379  29407   93786
 2.0      0.688904  0.311096  76354  34480  110834
 3.0      0.831532  0.168468   7384   1496    8880


In [1430]:
# 결측값 개수 확인
print("결측값 개수:", train['이식된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['이식된_배아_수'].fillna(0, inplace=True)
test['이식된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['이식된_배아_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 43. 미세주입_배아_이식_수

ICSI 배아 중 이식된 배아의 수

In [1431]:
value_counts_ratio(train, '미세주입_배아_이식_수')


미세주입_배아_이식_수별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.775604  0.224396  107370  31064  138434
 1.0      0.680553  0.319447   31628  14846   46474
 2.0      0.687657  0.312343   41038  18640   59678
 3.0      0.831643  0.168357    4263    863    5126


In [1432]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입_배아_이식_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입_배아_이식_수'].fillna(0, inplace=True)
test['미세주입_배아_이식_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입_배아_이식_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 44. 저장된_배아_수

환자를 위해 저장된 배아의 수

In [1433]:
value_counts_ratio(train, '저장된_배아_수')


저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.777446  0.222554  129468.0  37062.0  166530
 1.0      0.654636  0.345364   13331.0   7033.0   20364
 2.0      0.629471  0.370529   12512.0   7365.0   19877
 3.0      0.630277  0.369723    8510.0   4992.0   13502
 4.0      0.639628  0.360372    5914.0   3332.0    9246
 5.0      0.650494  0.349506    3888.0   2089.0    5977
 6.0      0.667477  0.332523    2746.0   1368.0    4114
 7.0      0.705350  0.294650    1872.0    782.0    2654
 8.0      0.714971  0.285029    1342.0    535.0    1877
 9.0      0.740482  0.259518     953.0    334.0    1287
 10.0     0.780849  0.219151     791.0    222.0    1013
 11.0     0.828076  0.171924     525.0    109.0     634
 12.0     0.841121  0.158879     450.0     85.0     535
 13.0     0.911616  0.088384     361.0     35.0     396
 14.0     0.936232 

In [1434]:
# 결측값 개수 확인
print("결측값 개수:", train['저장된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['저장된_배아_수'].fillna(0, inplace=True)
test['저장된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['저장된_배아_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 45. 미세주입_후_저장된_배아_수

ICSI 처리된 배아 중 저장된 배아의 수

In [1435]:
value_counts_ratio(train, '미세주입_후_저장된_배아_수')


미세주입_후_저장된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.755509  0.244491  154086.0  49864.0  203950
 1.0      0.653485  0.346515    8064.0   4276.0   12340
 2.0      0.624791  0.375209    7092.0   4259.0   11351
 3.0      0.633598  0.366402    4707.0   2722.0    7429
 4.0      0.645658  0.354342    3145.0   1726.0    4871
 5.0      0.656074  0.343926    1982.0   1039.0    3021
 6.0      0.673375  0.326625    1305.0    633.0    1938
 7.0      0.741986  0.258014     949.0    330.0    1279
 8.0      0.741419  0.258581     648.0    226.0     874
 9.0      0.768233  0.231767     474.0    143.0     617
 10.0     0.830835  0.169165     388.0     79.0     467
 11.0     0.851724  0.148276     247.0     43.0     290
 12.0     0.853282  0.146718     221.0     38.0     259
 13.0     0.925000  0.075000     185.0     15.0     200
 14.0     0.

In [1436]:
# 결측값 개수 확인
print("결측값 개수:", train['미세주입_후_저장된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['미세주입_후_저장된_배아_수'].fillna(0, inplace=True)
test['미세주입_후_저장된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['미세주입_후_저장된_배아_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 46. 해동된_배아_수

동결 해동된 배아의 수

In [1437]:
value_counts_ratio(train, '해동된_배아_수')


해동된_배아_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.731589  0.268411  153148.0  56188.0  209336
 1.0      0.784740  0.215260   10244.0   2810.0   13054
 2.0      0.761203  0.238797    9190.0   2883.0   12073
 3.0      0.794957  0.205043    4288.0   1106.0    5394
 4.0      0.778615  0.221385    2687.0    764.0    3451
 5.0      0.766529  0.233471    1484.0    452.0    1936
 6.0      0.752502  0.247498    1128.0    371.0    1499
 7.0      0.759463  0.240537     622.0    197.0     819
 8.0      0.698667  0.301333     524.0    226.0     750
 9.0      0.673966  0.326034     277.0    134.0     411
 10.0     0.740525  0.259475     254.0     89.0     343
 11.0     0.756410  0.243590     118.0     38.0     156
 12.0     0.673611  0.326389      97.0     47.0     144
 13.0     0.679012  0.320988      55.0     26.0      81
 14.0     0.720000 

In [1438]:
# 결측값 개수 확인
print("결측값 개수:", train['해동된_배아_수'].isnull().sum())

# 결측값을 0으로 채우기
train['해동된_배아_수'].fillna(0, inplace=True)
test['해동된_배아_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['해동된_배아_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 47. 해동_난자_수

동결 해동된 난자의 수

In [1439]:
value_counts_ratio(train, '해동_난자_수')


해동_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.737816  0.262184  183176.0  65092.0  248268
 1.0      0.923077  0.076923      24.0      2.0      26
 2.0      0.907692  0.092308      59.0      6.0      65
 3.0      0.907692  0.092308      59.0      6.0      65
 4.0      0.892308  0.107692     116.0     14.0     130
 5.0      0.859813  0.140187      92.0     15.0     107
 6.0      0.801170  0.198830     137.0     34.0     171
 7.0      0.711268  0.288732     101.0     41.0     142
 8.0      0.703196  0.296804     154.0     65.0     219
 9.0      0.780952  0.219048      82.0     23.0     105
 10.0     0.769231  0.230769      70.0     21.0      91
 11.0     0.761194  0.238806      51.0     16.0      67
 12.0     0.657143  0.342857      46.0     24.0      70
 13.0     0.812500  0.187500      26.0      6.0      32
 14.0     0.613636  

In [1440]:
# 결측값 개수 확인
print("결측값 개수:", train['해동_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['해동_난자_수'].fillna(0, inplace=True)
test['해동_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['해동_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 48. 수집된_신선_난자_수

신선 난자가 수집된 수

In [1441]:
value_counts_ratio(train, '수집된_신선_난자_수')


수집된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
-10.0     0.994169  0.005831    341.0      2.0    343
 0.0      0.759502  0.240498  40884.0  12946.0  53830
 1.0      0.948237  0.051763   4598.0    251.0   4849
 2.0      0.908907  0.091093   5847.0    586.0   6433
 3.0      0.867183  0.132817   7058.0   1081.0   8139
 4.0      0.823824  0.176176   8230.0   1760.0   9990
 5.0      0.790251  0.209749   9095.0   2414.0  11509
 6.0      0.773582  0.226418   9751.0   2854.0  12605
 7.0      0.746713  0.253287   9767.0   3313.0  13080
 8.0      0.724946  0.275054   9657.0   3664.0  13321
 9.0      0.717746  0.282254   9274.0   3647.0  12921
 10.0     0.697433  0.302567   8886.0   3855.0  12741
 11.0     0.682434  0.317566   8166.0   3800.0  11966
 12.0     0.671638  0.328362   7341.0   3589.0  10930
 13.0     0.664201  0.335799   6555.0   3314.0   9869
 14.0     0.667850  0.332150   6020.0   2994.0   9

In [1442]:
# 결측값 개수 확인
print("결측값 개수:", train['수집된_신선_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['수집된_신선_난자_수'].fillna(0, inplace=True)
test['수집된_신선_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['수집된_신선_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 49. 저장된_신선_난자_수

저장된 신선 난자의 수

In [1443]:
value_counts_ratio(train, '저장된_신선_난자_수')


저장된_신선_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.735478  0.264522  181755.0  65370.0  247125
 1.0      0.972028  0.027972     139.0      4.0     143
 2.0      0.989418  0.010582     187.0      2.0     189
 3.0      0.984772  0.015228     194.0      3.0     197
 4.0      0.980000  0.020000     196.0      4.0     200
 5.0      0.971292  0.028708     203.0      6.0     209
 6.0      0.980000  0.020000     196.0      4.0     200
 7.0      0.976048  0.023952     163.0      4.0     167
 8.0      0.994286  0.005714     174.0      1.0     175
 9.0      0.978571  0.021429     137.0      3.0     140
 10.0     0.993056  0.006944     143.0      1.0     144
 11.0     0.966102  0.033898     114.0      4.0     118
 12.0     0.981651  0.018349     107.0      2.0     109
 13.0     1.000000  0.000000      85.0      0.0      85
 14.0     0.9770

In [1444]:
# '저장된_신선_난자_수'와 '수집된_신선_난자_수'의 비율을 계산하여 새로운 파생 변수 생성
def calculate_ratio(row):
    if row['수집된_신선_난자_수'] == 0:
        return -1
    else:
        return round(row['저장된_신선_난자_수'] / row['수집된_신선_난자_수'], 1)

train['저장된_신선_난자_비율'] = train.apply(calculate_ratio, axis=1)
test['저장된_신선_난자_비율'] = test.apply(calculate_ratio, axis=1)

In [1445]:
pd.set_option('display.max_rows', 300)

In [1446]:
value_counts_ratio(train, '저장된_신선_난자_비율')


저장된_신선_난자_비율별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-1.0      0.771171  0.228829   46362  13757   60119
 0.0      0.728789  0.271211  140872  52424  193296
 0.1      0.692308  0.307692       9      4      13
 0.2      0.943396  0.056604      50      3      53
 0.3      0.953704  0.046296     103      5     108
 0.4      0.956897  0.043103     111      5     116
 0.5      0.963265  0.036735     236      9     245
 0.6      0.962567  0.037433     180      7     187
 0.7      0.989865  0.010135     293      3     296
 0.8      0.988914  0.011086     446      5     451
 0.9      0.997590  0.002410     414      1     415
 1.0      0.997129  0.002871    1042      3    1045


In [1447]:
# 결측값 개수 확인
print("결측값 개수:", train['저장된_신선_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['저장된_신선_난자_수'].fillna(0, inplace=True)
test['저장된_신선_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['저장된_신선_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 50. 혼합된_난자_수

정자와 혼합된 총 난자 수

In [1448]:
value_counts_ratio(train, '혼합된_난자_수')


혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
-10.0     0.994169  0.005831    341.0     2.0    343
 0.0      0.802525  0.197475  37189.0  9151.0  46340
 1.0      0.939333  0.060667   6054.0   391.0   6445
 2.0      0.892185  0.107815   7969.0   963.0   8932
 3.0      0.837596  0.162404   9268.0  1797.0  11065
 4.0      0.794485  0.205515  10805.0  2795.0  13600
 5.0      0.750387  0.249613  11637.0  3871.0  15508
 6.0      0.735944  0.264056  12029.0  4316.0  16345
 7.0      0.714617  0.285383  11411.0  4557.0  15968
 8.0      0.697967  0.302033  10644.0  4606.0  15250
 9.0      0.689570  0.310430   9725.0  4378.0  14103
 10.0     0.672589  0.327411   8745.0  4257.0  13002
 11.0     0.661372  0.338628   7662.0  3923.0  11585
 12.0     0.661103  0.338897   6734.0  3452.0  10186
 13.0     0.646627  0.353373   5636.0  3080.0   8716
 14.0     0.659124  0.340876   4981.0  2576.0   7557
 15.0     0.65693

In [1449]:
# 결측값 개수 확인
print("결측값 개수:", train['혼합된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['혼합된_난자_수'].fillna(0, inplace=True)
test['혼합된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['혼합된_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 51. 파트너_정자와_혼합된_난자_수

파트너 정자와 혼합된 난자의 수

In [1450]:
value_counts_ratio(train, '파트너_정자와_혼합된_난자_수')


파트너_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1        0        1  Total
variable                                             
-10.0     0.994169  0.005831    341.0      2.0    343
 0.0      0.781424  0.218576  48907.0  13680.0  62587
 1.0      0.939336  0.060664   5373.0    347.0   5720
 2.0      0.892287  0.107713   7207.0    870.0   8077
 3.0      0.837739  0.162261   8462.0   1639.0  10101
 4.0      0.796601  0.203399   9889.0   2525.0  12414
 5.0      0.751054  0.248946  10686.0   3542.0  14228
 6.0      0.737459  0.262541  11070.0   3941.0  15011
 7.0      0.717035  0.282965  10544.0   4161.0  14705
 8.0      0.698579  0.301421   9829.0   4241.0  14070
 9.0      0.690976  0.309024   9020.0   4034.0  13054
 10.0     0.674378  0.325622   8104.0   3913.0  12017
 11.0     0.660443  0.339557   7074.0   3637.0  10711
 12.0     0.661382  0.338618   6258.0   3204.0   9462
 13.0     0.646854  0.353146   5202.0   2840.0   8042
 14.0     0.659744  0.340256   4638.0   2392.

In [1451]:
# 결측값 개수 확인
print("결측값 개수:", train['파트너_정자와_혼합된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['파트너_정자와_혼합된_난자_수'].fillna(0, inplace=True)
test['파트너_정자와_혼합된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['파트너_정자와_혼합된_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 52. 기증자_정자와_혼합된_난자_수

기증자 정자와 혼합된 난자의 수

In [1452]:
value_counts_ratio(train, '기증자_정자와_혼합된_난자_수')


기증자_정자와_혼합된_난자_수별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.739207  0.260793  173137.0  61083.0  234220
 1.0      0.937415  0.062585     689.0     46.0     735
 2.0      0.887356  0.112644     772.0     98.0     870
 3.0      0.835700  0.164300     824.0    162.0     986
 4.0      0.770383  0.229617     926.0    276.0    1202
 5.0      0.738296  0.261704     962.0    341.0    1303
 6.0      0.715751  0.284249     977.0    388.0    1365
 7.0      0.690096  0.309904     864.0    388.0    1252
 8.0      0.691316  0.308684     804.0    359.0    1163
 9.0      0.671627  0.328373     677.0    331.0    1008
 10.0     0.647374  0.352626     604.0    329.0     933
 11.0     0.667485  0.332515     544.0    271.0     815
 12.0     0.652632  0.347368     434.0    231.0     665
 13.0     0.636060  0.363940     381.0    218.0     599
 14.0     0

In [1453]:
summarize_group(train, ['혼합된_난자_수','기증자_정자와_혼합된_난자_수', '파트너_정자와_혼합된_난자_수'])

Grouped by: 혼합된_난자_수, 기증자_정자와_혼합된_난자_수, 파트너_정자와_혼합된_난자_수

                     group  '임신_성공_여부' count     ratio  Total
0    (-10.0, -10.0, -10.0)                 2  0.005831    343
1          (0.0, 0.0, 0.0)              9151  0.197475  46340
2          (1.0, 0.0, 1.0)               346  0.060564   5713
3          (1.0, 1.0, 0.0)                45  0.061475    732
4          (2.0, 0.0, 2.0)               866  0.107364   8066
5          (2.0, 1.0, 1.0)                 0  0.000000      1
6          (2.0, 2.0, 0.0)                97  0.112139    865
7          (3.0, 0.0, 3.0)              1636  0.162157  10089
8          (3.0, 2.0, 1.0)                 0  0.000000      1
9          (3.0, 3.0, 0.0)               161  0.165128    975
10         (4.0, 0.0, 0.0)                 0  0.000000      5
11         (4.0, 0.0, 4.0)              2522  0.203387  12400
12         (4.0, 1.0, 3.0)                 1  0.500000      2
13         (4.0, 3.0, 1.0)                 0  0.000000      1
14         (

In [1454]:
# '기증자_정자와_혼합된_난자_수'와 '혼합된_난자_수'의 비율을 계산하여 새로운 파생 변수 생성
def calculate_donor_ratio(row):
    if row['혼합된_난자_수'] == 0:
        return -1
    else:
        return round(row['기증자_정자와_혼합된_난자_수'] / row['혼합된_난자_수'], 1)

# '파트너_정자와_혼합된_난자_수'와 '혼합된_난자_수'의 비율을 계산하여 새로운 파생 변수 생성
def calculate_partner_ratio(row):
    if row['혼합된_난자_수'] == 0:
        return -1
    else:
        return round(row['파트너_정자와_혼합된_난자_수'] / row['혼합된_난자_수'], 1)

train['기증자_정자_비율'] = train.apply(calculate_donor_ratio, axis=1)
test['기증자_정자_비율'] = test.apply(calculate_donor_ratio, axis=1)

train['파트너_정자_비율'] = train.apply(calculate_partner_ratio, axis=1)
test['파트너_정자_비율'] = test.apply(calculate_partner_ratio, axis=1)

In [1455]:
value_counts_ratio(train, '기증자_정자_비율')


기증자_정자_비율별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.810713  0.189287   42667.0   9962.0   52629
 0.0      0.723590  0.276410  135948.0  51932.0  187880
 0.1      0.000000  1.000000       0.0      1.0       1
 0.2      0.500000  0.500000       3.0      3.0       6
 0.3      0.714286  0.285714       5.0      2.0       7
 0.4      0.833333  0.166667      10.0      2.0      12
 0.5      0.681818  0.318182      30.0     14.0      44
 0.6      0.823529  0.176471      14.0      3.0      17
 0.7      0.571429  0.428571       8.0      6.0      14
 0.8      0.583333  0.416667       7.0      5.0      12
 0.9      1.000000  0.000000       4.0      0.0       4
 1.0      0.726683  0.273317   11422.0   4296.0   15718


In [1456]:
value_counts_ratio(train, '파트너_정자_비율')


파트너_정자_비율별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-1.0      0.810713  0.189287   42667.0   9962.0   52629
 0.0      0.721241  0.278759   11718.0   4529.0   16247
 0.1      1.000000  0.000000       4.0      0.0       4
 0.2      0.583333  0.416667       7.0      5.0      12
 0.3      0.571429  0.428571       8.0      6.0      14
 0.4      0.823529  0.176471      14.0      3.0      17
 0.5      0.681818  0.318182      30.0     14.0      44
 0.6      0.833333  0.166667      10.0      2.0      12
 0.7      0.714286  0.285714       5.0      2.0       7
 0.8      0.500000  0.500000       3.0      3.0       6
 0.9      0.000000  1.000000       0.0      1.0       1
 1.0      0.724053  0.275947  135652.0  51699.0  187351


In [1457]:
# 결측값 개수 확인
print("결측값 개수:", train['기증자_정자와_혼합된_난자_수'].isnull().sum())

# 결측값을 0으로 채우기
train['기증자_정자와_혼합된_난자_수'].fillna(0, inplace=True)
test['기증자_정자와_혼합된_난자_수'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['기증자_정자와_혼합된_난자_수'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 53. 난자_출처

난자 출처 정보

In [1458]:
value_counts_ratio(train, '난자_출처')


난자_출처별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
기증 제공     0.684571  0.315429   10795   4974   15769
본인 제공     0.741651  0.258349  173504  60439  233943
알 수 없음    0.871045  0.128955    5478    811    6289


In [1459]:
test['난자_출처'].value_counts()

난자_출처
본인 제공     82276
기증 제공      5493
알 수 없음     2176
-10         122
Name: count, dtype: int64

In [1460]:
# '난자_출처'에 따라 파생변수 생성
train['난자_출처_기증 제공'] = np.where(train['난자_출처'] == '기증 제공', 1, 0)
train['난자_출처_본인 제공'] = np.where(train['난자_출처'] == '본인 제공', 1, 0)
train['난자_출처_알 수 없음'] = np.where(train['난자_출처'] == '알 수 없음', 1, 0)

test['난자_출처_기증 제공'] = np.where(test['난자_출처'] == '기증 제공', 1, 0)
test['난자_출처_본인 제공'] = np.where(test['난자_출처'] == '본인 제공', 1, 0)
test['난자_출처_알 수 없음'] = np.where(test['난자_출처'] == '알 수 없음', 1, 0)

In [1461]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 85 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256001 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

난자_출처_알 수 없음 -> DI 시술임

In [1462]:
# '난자_출처_알 수 없음' 열을 제거
train = train.drop(columns=['난자_출처_알 수 없음', '난자_출처'])
test = test.drop(columns=['난자_출처_알 수 없음', '난자_출처'])

### 54. 정자_출처

정자 출처 정보

In [1463]:
value_counts_ratio(train, '정자_출처')


정자_출처별 임신_성공_여부 비율 및 갯수

                    0         1         0        1   Total
variable                                                  
-10          0.994169  0.005831     341.0      2.0     343
기증 제공        0.756013  0.243987   20401.0   6584.0   26985
미할당          0.893443  0.106557     109.0     13.0     122
배우자 및 기증 제공  1.000000  0.000000      14.0      0.0      14
배우자 제공       0.739484  0.260516  169253.0  59627.0  228880


In [1464]:
test['정자_출처'].value_counts()

정자_출처
배우자 제공         80462
기증 제공           9445
-10              122
미할당               33
배우자 및 기증 제공        5
Name: count, dtype: int64

In [1465]:
# '정자_출처'에 따라 파생변수 생성
train['정자_출처_배우자 제공'] = np.where(train['정자_출처'].isin(['배우자 제공', '배우자 및 기증 제공']), 1, 0)
train['정자_출처_기증 제공'] = np.where(train['정자_출처'].isin(['기증 제공', '배우자 및 기증 제공']), 1, 0)
train['정자_출처_미할당'] = np.where(train['정자_출처'] == '미할당', 1, 0)

test['정자_출처_배우자 제공'] = np.where(test['정자_출처'].isin(['배우자 제공', '배우자 및 기증 제공']), 1, 0)
test['정자_출처_기증 제공'] = np.where(test['정자_출처'].isin(['기증 제공', '배우자 및 기증 제공']), 1, 0)
test['정자_출처_미할당'] = np.where(test['정자_출처'] == '미할당', 1, 0)

In [1466]:
train = train.drop(columns=['정자_출처'])
test = test.drop(columns=['정자_출처'])

### 55. 난자_기증자_나이

난자 기증자의 등록 당시 나이

In [1467]:
value_counts_ratio(train, '난자_기증자_나이')


난자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
만20세 이하   0.738095  0.261905     217     77     294
만21-25세   0.670094  0.329906    1564    770    2334
만26-30세   0.651728  0.348272    3243   1733    4976
만31-35세   0.694785  0.305215    4423   1943    6366
알 수 없음    0.745070  0.254930  180330  61701  242031


In [1468]:
# '난자_기증자_나이' 변수 값 변환
age_mapping = {
    '만20세 이하': 20,
    '만21-25세': 23,
    '만26-30세': 28,
    '만31-35세': 32,
    '알 수 없음': -1
}

train['난자_기증자_나이'] = train['난자_기증자_나이'].replace(age_mapping)
test['난자_기증자_나이'] = test['난자_기증자_나이'].replace(age_mapping)

In [1469]:
value_counts_ratio(train, '난자_기증자_나이')


난자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
-1        0.745070  0.254930  180330  61701  242031
 20       0.738095  0.261905     217     77     294
 23       0.670094  0.329906    1564    770    2334
 28       0.651728  0.348272    3243   1733    4976
 32       0.694785  0.305215    4423   1943    6366


In [1470]:
# 결측값 개수 확인
print("결측값 개수:", train['난자_기증자_나이'].isnull().sum())

결측값 개수: 0


### 56. 정자_기증자_나이

정자 기증자의 등록 당시 나이

In [1471]:
value_counts_ratio(train, '정자_기증자_나이')


정자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
만20세 이하   0.790066  0.209934     843    224    1067
만21-25세   0.758694  0.241306    4298   1367    5665
만26-30세   0.751633  0.248367    3798   1255    5053
만31-35세   0.751222  0.248778    3687   1221    4908
만36-40세   0.753887  0.246113    3976   1298    5274
만41-45세   0.756510  0.243490    2905    935    3840
알 수 없음    0.739680  0.260320  170270  59924  230194


In [1472]:
# '정자_기증자_나이' 변수 값 변환
age_mapping = {
    '만20세 이하': 20,
    '만21-25세': 23,
    '만26-30세': 28,
    '만31-35세': 32,
    '만36-40세': 38,
    '만41-45세': 43,
    '알 수 없음': -1
}

train['정자_기증자_나이'] = train['정자_기증자_나이'].replace(age_mapping)
test['정자_기증자_나이'] = test['정자_기증자_나이'].replace(age_mapping)

In [1473]:
value_counts_ratio(train, '정자_기증자_나이')


정자_기증자_나이별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10       0.994169  0.005831     341      2     343
-1        0.739680  0.260320  170270  59924  230194
 20       0.790066  0.209934     843    224    1067
 23       0.758694  0.241306    4298   1367    5665
 28       0.751633  0.248367    3798   1255    5053
 32       0.751222  0.248778    3687   1221    4908
 38       0.753887  0.246113    3976   1298    5274
 43       0.756510  0.243490    2905    935    3840


In [1474]:
# 결측값 개수 확인
print("결측값 개수:", train['정자_기증자_나이'].isnull().sum())

결측값 개수: 0


### 57. 동결_배아_사용_여부

동결 배아가 사용되었는지 여부

In [1475]:
value_counts_ratio(train, '동결_배아_사용_여부')


동결_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.731811  0.268189  153384  56211  209595
 1.0      0.770621  0.229379   30915   9202   40117


In [1476]:
# 결측값 개수 확인
print("결측값 개수:", train['동결_배아_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['동결_배아_사용_여부'].fillna(-1, inplace=True)
test['동결_배아_사용_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['동결_배아_사용_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 58. 신선_배아_사용_여부

신선 배아가 사용되었는지 여부

In [1477]:
value_counts_ratio(train, '신선_배아_사용_여부')


신선_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.770437  0.229563   30752   9163   39915
 1.0      0.731884  0.268116  153547  56250  209797


In [1478]:
# 결측값 개수 확인
print("결측값 개수:", train['신선_배아_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['신선_배아_사용_여부'].fillna(-1, inplace=True)
test['신선_배아_사용_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['신선_배아_사용_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 59. 기증_배아_사용_여부

기증 배아가 사용되었는지 여부

In [1479]:
value_counts_ratio(train, '기증_배아_사용_여부')


기증_배아_사용_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.738698  0.261302  182646  64608  247254
 1.0      0.672498  0.327502    1653    805    2458


In [1480]:
# 결측값 개수 확인
print("결측값 개수:", train['기증_배아_사용_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['기증_배아_사용_여부'].fillna(-1, inplace=True)
test['기증_배아_사용_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['기증_배아_사용_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


In [1481]:
summarize_group(train, ['신선_배아_사용_여부', '동결_배아_사용_여부', '기증_배아_사용_여부'])

Grouped by: 신선_배아_사용_여부, 동결_배아_사용_여부, 기증_배아_사용_여부

                   group  '임신_성공_여부' count     ratio   Total
0  (-10.0, -10.0, -10.0)                 2  0.005831     343
1     (-1.0, -1.0, -1.0)               811  0.128955    6289
2        (0.0, 0.0, 0.0)                 1  0.333333       3
3        (0.0, 1.0, 0.0)              8914  0.229004   38925
4        (0.0, 1.0, 1.0)               248  0.251266     987
5        (1.0, 0.0, 0.0)             55653  0.267407  208121
6        (1.0, 0.0, 1.0)               557  0.378654    1471
7        (1.0, 1.0, 0.0)                40  0.195122     205


### 60. 대리모_여부

환자가 대리모로 활동했는지 여부

In [1482]:
value_counts_ratio(train, '대리모_여부')


대리모_여부별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.738165  0.261835  183558  65110  248668
 1.0      0.709770  0.290230     741    303    1044


In [1483]:
# 결측값 개수 확인
print("결측값 개수:", train['대리모_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['대리모_여부'].fillna(-1, inplace=True)
test['대리모_여부'].fillna(-1, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['대리모_여부'].isnull().sum())

결측값 개수: 6289
결측값 개수: 0


### 61. PGD_시술_여부

해당 주기가 PGD 치료로 분류되는지 여부

In [1484]:
value_counts_ratio(train, 'PGD_시술_여부')


PGD_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
-10.0     0.994169  0.005831   341    2    343
 1.0      0.720514  0.279486  1570  609   2179


In [1485]:
train['PGD_시술_여부'].isnull().sum()

253822

In [1486]:
# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGD_시술_여부'].fillna(0, inplace=True)
test['PGD_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGD_시술_여부'].isnull().sum())

결측값 개수: 253822
결측값 개수: 0


### 62. PGS_시술_여부

해당 주기가 PGS 치료로 분류되는지 여부

In [1487]:
value_counts_ratio(train, 'PGS_시술_여부')


PGS_시술_여부별 임신_성공_여부 비율 및 갯수

                 0         1     0    1  Total
variable                                      
-10.0     0.994169  0.005831   341    2    343
 1.0      0.793675  0.206325  1531  398   1929


In [1488]:
# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

# 결측값을 0으로 채우기
train['PGS_시술_여부'].fillna(0, inplace=True)
test['PGS_시술_여부'].fillna(0, inplace=True)

# 결측값 개수 확인
print("결측값 개수:", train['PGS_시술_여부'].isnull().sum())

결측값 개수: 254072
결측값 개수: 0


In [1489]:
summarize_group(train, ['PGD_시술_여부', 'PGS_시술_여부'])

Grouped by: PGD_시술_여부, PGS_시술_여부

            group  '임신_성공_여부' count     ratio   Total
0  (-10.0, -10.0)                 2  0.005831     343
1      (0.0, 0.0)             65217  0.258908  251893
2      (0.0, 1.0)               398  0.206325    1929
3      (1.0, 0.0)               609  0.279486    2179


### 63. 난자_채취_경과일

첫 시점으로부터 난자 채취까지 경과한 일수

In [1490]:
value_counts_ratio(train, '난자_채취_경과일')


난자_채취_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1   Total
variable                                           
-10.0     0.994169  0.005831     341      2     343
 0.0      0.735741  0.264259  146063  52462  198525


In [1491]:
print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

train['난자_채취_경과일'].fillna(-1, inplace=True)
test['난자_채취_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_채취_경과일'].isnull().sum())

결측값 개수: 57476
결측값 개수: 0


### 64. 난자_해동_경과일

첫 시점으로부터 난자 해동까지 경과한 일수

In [1492]:
value_counts_ratio(train, '난자_해동_경과일')


난자_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1       0      1  Total
variable                                          
-10.0     0.994169  0.005831   341.0    2.0    343
 0.0      0.777390  0.222610  1114.0  319.0   1433
 1.0      1.000000  0.000000     2.0    0.0      2


In [1493]:
train['난자_해동_경과일'].isnull().sum()

254566

In [1494]:
print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

train['난자_해동_경과일'].fillna(-1, inplace=True)
test['난자_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_해동_경과일'].isnull().sum())

결측값 개수: 254566
결측값 개수: 0


### 65. 난자_혼합_경과일

첫 시점으로부터 난자와 정자 혼합까지 경과한 일수

In [1495]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
 0.0      0.722879  0.277121  145937.0  55946.0  201883
 1.0      0.846311  0.153689     413.0     75.0     488
 2.0      0.754902  0.245098      77.0     25.0     102
 3.0      0.720588  0.279412      49.0     19.0      68
 4.0      0.600000  0.400000       3.0      2.0       5
 5.0      0.791667  0.208333      19.0      5.0      24
 6.0      1.000000  0.000000       8.0      0.0       8
 7.0      1.000000  0.000000       1.0      0.0       1


In [1496]:
train['난자_혼합_경과일'].isnull().sum()

53422

In [1497]:
print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

train['난자_혼합_경과일'].fillna(-1, inplace=True)
test['난자_혼합_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['난자_혼합_경과일'].isnull().sum())

결측값 개수: 53422
결측값 개수: 0


In [1498]:
value_counts_ratio(train, '난자_혼합_경과일')


난자_혼합_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
-1.0      0.809966  0.190034   43270.0  10152.0   53422
 0.0      0.722879  0.277121  145937.0  55946.0  201883
 1.0      0.846311  0.153689     413.0     75.0     488
 2.0      0.754902  0.245098      77.0     25.0     102
 3.0      0.720588  0.279412      49.0     19.0      68
 4.0      0.600000  0.400000       3.0      2.0       5
 5.0      0.791667  0.208333      19.0      5.0      24
 6.0      1.000000  0.000000       8.0      0.0       8
 7.0      1.000000  0.000000       1.0      0.0       1


### 66. 배아_이식_경과일

첫 시점으로부터 배아 이식까지 경과한 일수

In [1499]:
value_counts_ratio(train, '배아_이식_경과일')


배아_이식_경과일별 임신_성공_여부 비율 및 갯수

                 0         1      0      1  Total
variable                                         
-10.0     0.994169  0.005831    341      2    343
 0.0      0.749016  0.250984  18649   6249  24898
 1.0      0.812954  0.187046   4920   1132   6052
 2.0      0.787519  0.212481  27623   7453  35076
 3.0      0.741234  0.258766  42933  14988  57921
 4.0      0.655563  0.344437   2952   1551   4503
 5.0      0.595541  0.404459  48511  32946  81457
 6.0      0.699964  0.300036   1941    832   2773
 7.0      0.588889  0.411111     53     37     90


In [1500]:
print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

train['배아_이식_경과일'].fillna(-1, inplace=True)
test['배아_이식_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_이식_경과일'].isnull().sum())

결측값 개수: 43231
결측값 개수: 0


### 67. 배아_해동_경과일

첫 시점으로부터 배아 해동까지 경과한 일수

In [1501]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1        0       1  Total
variable                                            
-10.0     0.994169  0.005831    341.0     2.0    343
 0.0      0.770356  0.229644  30654.0  9138.0  39792
 1.0      0.837838  0.162162     31.0     6.0     37
 2.0      0.840659  0.159341    153.0    29.0    182
 3.0      0.890244  0.109756    146.0    18.0    164
 4.0      1.000000  0.000000     12.0     0.0     12
 5.0      0.785714  0.214286    110.0    30.0    140
 6.0      0.833333  0.166667     25.0     5.0     30
 7.0      1.000000  0.000000      3.0     0.0      3


In [1502]:
test['배아_해동_경과일'].value_counts()

배아_해동_경과일
 0.0     13744
-10.0      122
 3.0        59
 2.0        57
 5.0        55
 1.0        17
 6.0         7
 4.0         4
Name: count, dtype: int64

In [1503]:
print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

train['배아_해동_경과일'].fillna(-1, inplace=True)
test['배아_해동_경과일'].fillna(-1, inplace=True)

print("결측값 개수:", train['배아_해동_경과일'].isnull().sum())

결측값 개수: 215641
결측값 개수: 0


In [1504]:
value_counts_ratio(train, '배아_해동_경과일')


배아_해동_경과일별 임신_성공_여부 비율 및 갯수

                 0         1         0        1   Total
variable                                               
-10.0     0.994169  0.005831     341.0      2.0     343
-1.0      0.735681  0.264319  158643.0  56998.0  215641
 0.0      0.770356  0.229644   30654.0   9138.0   39792
 1.0      0.837838  0.162162      31.0      6.0      37
 2.0      0.840659  0.159341     153.0     29.0     182
 3.0      0.890244  0.109756     146.0     18.0     164
 4.0      1.000000  0.000000      12.0      0.0      12
 5.0      0.785714  0.214286     110.0     30.0     140
 6.0      0.833333  0.166667      25.0      5.0      30
 7.0      1.000000  0.000000       3.0      0.0       3


In [1505]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 85 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256001 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

In [1506]:
import pandas as pd
import numpy as np

# 파생 변수 생성 함수
def create_features(df):
    # 총 배아 생성 비율
    df['생성_배아_대비_이식_배아_비율'] = np.where(df['총_생성_배아_수'] != 0, df['이식된_배아_수'] / df['총_생성_배아_수'], -1)

    # 미세주입 배아 이식 비율
    df['ICSI_배아_이식_비율'] = np.where(df['미세주입에서_생성된_배아_수'] != 0, df['미세주입_배아_이식_수'] / df['미세주입에서_생성된_배아_수'], -1)

    # 이식된 배아 중 ICSI 배아가 이식된 비율
    df['이식된_배아중_ICSI_배아_비율'] = np.where(df['이식된_배아_수'] != 0, df['미세주입에서_생성된_배아_수'] / df['이식된_배아_수'], -1)

    # 저장된 배아 비율
    df['저장된_배아_비율'] = np.where(df['총_생성_배아_수'] != 0, df['저장된_배아_수'] / df['총_생성_배아_수'], -1)
    
    # 소수점 둘째 자리까지 반올림
    df = df.round(2)
    
    # inf, -inf 값을 -1로 대체
    df.replace([np.inf, -np.inf], -1, inplace=True)
    
    return df

# 파생 변수 생성
train = create_features(train)
test = create_features(test)

### 68. 임신_성공_여부

해당 시술로부터 임신 성공 여부

In [1507]:
train['임신_성공_여부'].value_counts()

임신_성공_여부
0    190118
1     66226
Name: count, dtype: int64

## 변수 상태 확인

In [1508]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256344 entries, 0 to 256343
Data columns (total 89 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     256344 non-null  object 
 1   시술_당시_나이               256001 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  256344 non-null  float64
 3   배란_자극_여부               256344 non-null  int64  
 4   단일_배아_이식_여부            256344 non-null  float64
 5   착상_전_유전_검사_사용_여부       256344 non-null  float64
 6   착상_전_유전_진단_사용_여부       256344 non-null  float64
 7   남성_주_불임_원인             256344 non-null  int64  
 8   남성_부_불임_원인             256344 non-null  int64  
 9   여성_주_불임_원인             256344 non-null  int64  
 10  여성_부_불임_원인             256344 non-null  int64  
 11  부부_주_불임_원인             256344 non-null  int64  
 12  부부_부_불임_원인             256344 non-null  int64  
 13  불명확_불임_원인              256344 non-null  int64  
 14  불임_원인_-_난관_질환          256344 non-nu

In [1509]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 88 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     90067 non-null  object 
 1   시술_당시_나이               89945 non-null  float64
 2   임신_시도_또는_마지막_임신_경과_연수  90067 non-null  float64
 3   배란_자극_여부               90067 non-null  int64  
 4   단일_배아_이식_여부            90067 non-null  float64
 5   착상_전_유전_검사_사용_여부       90067 non-null  float64
 6   착상_전_유전_진단_사용_여부       90067 non-null  float64
 7   남성_주_불임_원인             90067 non-null  int64  
 8   남성_부_불임_원인             90067 non-null  int64  
 9   여성_주_불임_원인             90067 non-null  int64  
 10  여성_부_불임_원인             90067 non-null  int64  
 11  부부_주_불임_원인             90067 non-null  int64  
 12  부부_부_불임_원인             90067 non-null  int64  
 13  불명확_불임_원인              90067 non-null  int64  
 14  불임_원인_-_난관_질환          90067 non-null  int64  
 15  불임

In [1510]:
# # 결측값을 -1 으로 채우기
# train.fillna(-1, inplace=True)
# test.fillna(-1, inplace=True)

## 데이터 저장

In [1511]:
# CSV 파일 저장
train.to_csv('../data/Total_train_dataset_51.csv', index=False, encoding='utf-8-sig')
test.to_csv('../data/Total_test_dataset_51.csv', index=False, encoding='utf-8-sig')

.